In [ ]:

import numpy as np

class ConvolutionLayer:
  # A Convolution layer

    # generate random matrix
    def generateRandomMatrix(self): 
        rand = self.channel * self.width * self.height;
        matrix = np.zeros((self.channel, self.width, self.height))
        for i in range (self.channel):
            for j in range (self.width):
                for k in range (self.height):
                    matrix[i][j][k] = np.random.normal(loc = 0, scale = np.sqrt(1. / rand))
        return matrix

    # create filter matrix
    def createFilterMatrix(self):
        for i in range(self.num_filters):
            # initialize filter matrix with random
            self.weights[i] = self.generateRandomMatrix()
        return 

    # initialization function to create convolutional layer instance
    def __init__(self, inputs_channel, num_filters, width, height, padding, stride, learning_rate):

        # number of convolutional filter as defined, it is required to be at least 1
        if num_filters <=0: 
             print("invalid filter number is provided.")
             return
        else:
            self.num_filters = num_filters
            # bias matrix is set to all zero initially
            self.bias = np.zeros((self.num_filters,1))
        # number of image channels. In the project, all image channel is decided to be 1 for gray image
        if inputs_channel <=0: 
             print("invalid channel number is provided.")
             return
        else:
            self.channel = inputs_channel
        # width of input images
        if width <=0: 
             print("invalid width of matrix is provided.")
             return
        else:
            self.width = width
        # height of input images
        if height <=0: 
             print("invalid height of matrix is provided.")
             return
        else:
            self.height = height
            # create random convolutional matrix
            self.weights = np.zeros((self.num_filters, self.channel, self.width, self.height))
        # step size of convolutional matrix's sliding step
        if stride <=0: 
             print("Value of stride needs to be greater than 0 to be meaningful.")
             return
        else:
            self.stride = stride
        # padding for result
        if padding < 0: 
             print("padding can't be negative value.")
             return
        else:
            self.padding = padding
        # learning rate for gradient descent study
        if learning_rate  < 0: 
            print("Please provide a positive learning rate")
        else:
            self.lr = learning_rate

    
    def paddingInclusion(self, inputs):
        # get input image's shape and calculate result image shape including padding size
        w = inputs.shape[0]
        h = inputs.shape[1]
        # new shape of matrix after considering padding
        new_w = 2 * self.padding + w
        new_h = 2 * self.padding + h
        out = np.zeros((new_w, new_h))
        # fill those zero spots by input values and rest will be left as it is
        for i in range(new_w):
            for j in range(new_h):
                if i >= self.padding and i < self.padding + w and j >= self.padding and j < self.padding + h:
                    out[i][j] = inputs[i-self.padding][j-self.padding]
        return out

    def sumResultOfMatrix(self, changedMatrix):
        f = changedMatrix.shape[0]
        w = changedMatrix.shape[1]
        h = changedMatrix.shape[2]
        result = 0
        for i in range(f):
            for j in range(w):
                for k in range(h):
                    result += changedMatrix[i][j][k]
        return result


    def calculateForwardMatrix(self, new_width, new_height):
        # corner check
        if new_width > 0 and new_height > 0:
            forward_matrix = np.zeros((self.num_filters, new_width, new_height))
            for f in range(self.num_filters):
                for w in range(new_width):
                    for h in range(new_height):
                        changedMatrix = self.inputs[:,w:w+self.width,h:h+self.height]*self.weights[f,:,:,:]
                        # sum up result to form forward matrix
                        forward_matrix[f,w,h]= self.sumResultOfMatrix(changedMatrix)+self.bias[f]
            return forward_matrix
        else:
            print("either width or height is invalid")
            return

    def forward(self, inputs):
        # get input shape, height, width and channel
        channel = inputs.shape[0]
        padding = 2 * self.padding
        stride = self.stride
        process_width = padding + inputs.shape[1]
        process_height = padding + inputs.shape[2]
        self.inputs = np.zeros((channel, process_width, process_height))
        for ch in range(channel):
            # process padding
            self.inputs[ch] = self.paddingInclusion(inputs[ch])
        new_width = (process_width - self.width)//stride + 1
        new_height = (process_height - self.height)//stride + 1
        return self.calculateForwardMatrix(new_width, new_height)

    def backward(self, dy):
        inputs = self.inputs
        weights = self.weights
        bias = self.bias
        src = np.zeros((inputs.shape[0], inputs.shape[1], inputs.shape[2]))
        wei = np.zeros((weights.shape[0], weights.shape[1], weights.shape[2]))
        bi = np.zeros((bias.shape[0], bias.shape[1], bias.shape[2]))

        fil, width, height = dy.shape
        if fil > 0 and width > 0 and height > 0:
            for f in range(fil):
                for w in range(width):
                    for h in range(height):
                        wei[f]+=dy[f,w,h]*self.inputs[:,w:w+self.width,h:h+self.height]
                        src[:,w:w+self.width,h:h+self.height]+=dy[f,w,h]*self.weights[f,:,:,:]
        else:
            return

        for f in range(fil):
            sum = 0;
            target_matrix = dy[f]
            for i in range (target_matrix.shape[0]):
                for j in range (target_matrix.shape[1]):
                    sum += target_matrix[i][j]
            bi[f] = sum

        self.weights -= self.lr * wei
        self.bias -= self.lr * bi
        return src
        

In [ ]:
class ReLu:
    def forward(self, inputs):
        self.inputs = inputs
        relu = inputs.copy()
        # change the value that is less than 0 to 0
        relu[relu < 0] = 0
        return relu

    def backward(self, dy):
        inputs = dy.copy()
        inputs[self.inputs < 0] = 0
        return inputs

In [ ]:
class MaxPoolingLayer:
    def __init__(self, width, height, stride):
        if width > 0:
            self.width = width
        else:
            sys.exit("width cannot be negative") 
        if height > 0:
            self.height = height
        else:
            sys.exit("height cannot be negative") 
        if stride > 0:
            self.stride = stride
        else:
            sys.exit("stride cannot be negative") 

    def new_demension(self, current, old, stride):
        if current > 0 and old > 0 and stride > 0:
            return (current - old) // stride + 1
        else:
            return 0

    def array_slice(self, i1, i2, j1, j2, k1, k2):
        return self.inputs[i1, j1:j2, k1:k2]

    def arr_max(self, arr):
        result = np.max(arr[0])
        for v in arr:
            result = max(np.max(v), result)
        return result

    def forward_out(self, c_len, new_width, new_height):
        out = 0
        if c_len >= 0 and new_width >= 0 and new_height >= 0:
            out = np.zeros((c_len, new_width, new_height))
        else:
            sys.exit("Inputs should not be negative") 
        c = 0
        w = 0
        h = 0
        flag_c = True
        flag_w = True
        flag_h = True
        while flag_c:
            while flag_w:
                while flag_h:
                    new_c = c
                    new_c_2 = c + 1
                    new_w_1 = self.stride * w
                    new_w_2 = new_w_1 + self.width
                    new_h_1 = h * self.stride
                    new_h_2 = new_h_1 + self.height
                    result = self.array_slice(new_c, new_c_2, new_w_1, new_w_2, new_h_1, new_h_2)
                    out[c, w, h] = self.arr_max(result)
                    h += 1
                    flag_h = h < new_height
                w += 1
                flag_w = w < new_width
            c += 1
            flag_c = c < c_len
        return out

    def forward(self, inputs):
        if inputs is not None:
            self.inputs = inputs
        else:
            sys.exit("Inputs should not be None") 

        c_len, w_len, h_len = inputs.shape
        new_width = self.new_demension(w_len, self.width, self.stride)
        new_height = self.new_demension(h_len, self.height, self.stride)
        result = self.forward_out(c_len,new_width, new_height)
        if result is not None:
            return result
        else:
            sys.exit("error: output is none") 

    def index_to_tuple(index, m, n):
        x = int(index / n)
        y = index - n * x
        return x, y

    def backward_out(self, c_len, w_len, h_len, delta_y):
        dx = np.zeros(self.inputs.shape)
        c = 0
        w = 0
        h = 0
        flag_c = True
        flag_w = True
        flag_h = True
        while flag_c:
            while flag_w:
                while flag_h:
                    new_c = c
                    new_c_2 = c + 1
                    new_w_1 = w
                    new_w_2 = new_w_1 + self.width
                    new_h_1 = h
                    new_h_2 = new_h_1 + self.height
                    result = self.array_slice(new_c, new_c_2, new_w_1, new_w_2, new_h_1, new_h_2)
                    x, y = self.index_to_tuple(np.argmax(result), new_w_1, new_h_1)
                    new_w_1 += x
                    new_h_1 += y
                    dx[new_c, new_w_1, new_h_1] = delta_y[c, w // self.width, h // self.height]
                    h += self.height
                    flag_h = h < h_len
                w += self.width
                flag_w = w < w_len
            c += 1
            flag_c = c < c_len

        return dx

    def backward(self, delta_y):
        c_len, w_len, h_len = self.inputs.shape
        if c_len < 0 or w_len < 0 or h_len < 0:
            sys.exit("Inputs should not be negative")
        result = self.backward_out(c_len, w_len, h_len, delta_y)
        return result

In [ ]:
def cross_entropy(inputs, labels):
    return -np.log(np.sum(labels.reshape(1,labels.shape[0])*inputs))

In [ ]:
class Flatten:
    def forward(self, i):
        self.c_len = i.shape[0]
        self.w_len = i.shape[1]
        self.h_len = i.shape[2]
        return i.reshape(1, i.shape[0]*i.shape[1]*i.shape[2])
    def backward(self, y):
        return y.reshape(self.c_len, self.w_len, self.h_len)
    def extract(self): 
        return

In [ ]:
class FullyConnectedLayer:

    def __init__(self, num_inputs, num_outputs, learning_rate):
        if num_inputs < 0 or num_outputs < 0:
            print("invalid values for num_inputs and num_outputs")
            return
        self.weights = 0.01*np.random.rand(num_inputs, num_outputs)
        self.bias = np.zeros((num_outputs, 1))
        if learning_rate == 0:
            print("invalid values for learning_rate")
            return
        self.lr = learning_rate

    def forward(self, inputs):
        self.inputs = inputs
        return np.dot(self.inputs, self.weights) + self.bias.T

    def backward(self, dy):
        if dy.shape[0] == self.inputs.shape[0]:
            dy = dy.T
        dw = dy.dot(self.inputs)
        db = np.sum(dy, axis=1, keepdims=True)
        dx = np.dot(dy.T, self.weights.T)

        self.weights -= self.lr * dw.T
        self.bias -= self.lr * db

        return dx

In [ ]:
class Softmax:
    def forward(self, inputs):
        tmp = np.exp(inputs, dtype=np.float)
        self.out = tmp/np.sum(tmp)
        return self.out
    def backward(self, dy):
        return self.out.T - dy.reshape(dy.shape[0],1)


In [ ]:
class CNN:
    def __init__(self, _layers):
        self.layers = _layers
        self.lay_num = len(layers)
        
    def get_data(self, trn_X, trn_y, i, batch):
        flag = True
        len_0 = trn_X.shape[0] 
        if batch > 0:
            self.b = batch
            flag = (len_0 - batch - i > 0)
        else:
            sys.exit("batch cannot be negative")  
        i2 = len_0
        if flag:
            i2 = i + batch
        return trn_X[i:i2], trn_y[i:i2]

    def forward(self, input_x):
        for l in range(self.lay_num):
            output = self.layers[l].forward(input_x)
            input_x = output
        return output

    def backward(self, output_y):
        l = lay_num
        flag = (l >= 0)
        out = output_y
        while flag:
            out = self.layers[l].backward(out)
            l -= 1
            flag = (l >= 0)

    def train(self, training_data, training_label, batch_size, epoch):
        acc_sum = 0
        X0_len = training_data.shape[0]
        for e in range(epoch):
            batch_index = 0
            flag = (batch_index < X0_len)
            while flag:
                # get the data
                data, label = self.get_data(training_data, training_label, i, batch_size) 
                loss = 0
                acc = 0

                for b in range(len(data)):
                    x = data[b]
                    y = label[b]

                    # forward
                    output = self.forward(x)

                    loss += cross_entropy(output, y)

                    predict_y = np.argmax(output)
                    real_y = np.argmax(y)
                    flag = (predict_y != real_y)

                    if flag:
                        acc += 0
                        acc_sum += 0
                    else:
                        acc += 1
                        acc_sum += 1
                    #backward
                    self.backward(y)
                batch_index += batch_size
                flag = (batch_index < X0_len)


                # result
                loss /= batch_size
                batch_acc = float(acc) / float(batch_size)
                training_acc = float(acc_sum) / float((batch_index + batch_size) * (e + 1))
 
    def predict(self, x):
        return self.forward(x)

    def test(self, data, label, test_size):
        total_acc = 0
        for i in range(test_size):
            x = data[i]
            y = label[i]

            predict_y = np.argmax(self.predict(x))
            real_y = np.argmax(y)
            if predict_y != real_y:
                total_acc += 0
            else:
                total_acc += 1

        return  float(total_acc) / float(test_size)

   

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 17 15:03:57 2020

@author: MoHan
"""

# libraries needed
import os

import math

import matplotlib.pyplot as plt
import numpy 

# note: if tensorflow is not install, run "pip install --upgrade tensorflow"
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image
import random

test_dir = "./dataset/test_set"
train_dir = "./dataset/training_set"

train_dir_cats = train_dir + "/cats"
train_dir_dogs = train_dir + "/dogs"
test_dir_cats = test_dir + "/cats"
test_dir_dogs = test_dir + "/dogs"

train_data = []
train_data_label = []
test_data = []
test_data_label = []

# Only transformed to gray pic
def normal_transform (imgpath):
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (28,28))
    return Image.fromarray(img)

# Preprocessed using gaussian_canny
def gaussian_canny_transform (imgpath):
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gaussian = cv2.GaussianBlur(img, (3,3), 0)
    gaussian = gaussian.astype(numpy.uint8)
    canny = cv2.Canny(gaussian, 50, 50)
    canny = cv2.resize(canny, (28,28))
    return Image.fromarray(canny)

# Reading training data
def read_training_data(train_data, train_data_label, dir, label):
    for filename in os.listdir(dir):
        imgpath = dir + "/" + filename
        img = normal_transform(imgpath)
        train_data.append([numpy.asarray(img)])
        train_data_label.append((label))

# Reading testing data
def read_testing_data(test_data, test_data_label, dir, label):
    for filename in os.listdir(dir):
        imgpath = dir + "/" + filename
        img = normal_transform(imgpath)
        test_data.append([numpy.asarray(img)])
        test_data_label.append((label))

#read gray images into train_data and train_data_label
read_training_data(train_data, train_data_label, train_dir_cats, [0,1])
#train_data =train_data[0:250]
#train_data_label =train_data_label[0:250]
read_training_data(train_data, train_data_label, train_dir_dogs, [1,0])
#train_data =train_data[0:500]
#train_data_label =train_data_label[0:500]
for i in range(0,len(train_data)//2,2):
               tmp=train_data[i]
               train_data[i] =train_data[len(train_data)-1-i]
               train_data[len(train_data)-1-i] =tmp
               #print(train_data_label[i])
               tlabel = train_data_label[i]
               train_data_label[i] = train_data_label[len(train_data)-1-i]
               train_data_label[len(train_data)-1-i] =  tlabel
               #print(train_data_label[i])
               



read_testing_data(test_data, test_data_label, test_dir_cats, [0,1])

read_testing_data(test_data, test_data_label, test_dir_dogs, [1,0])
for i in range(0,len(test_data)//2,3):
               tmp = test_data[i]
               test_data[i] =test_data[len(test_data)-1-i]
               test_data[len(test_data)-1-i] =tmp
               tlabel = test_data_label[i]
               test_data_label[i] = test_data_label[len(test_data)-1-i]
               test_data_label[len(test_data)-1-i] =  tlabel
               i=i+1
train_data = train_data[0:500]
train_data_label=train_data_label[0:500]
test_data=test_data[1000:1200]
test_data_label =test_data_label[1000:1200]

train_data = numpy.array(train_data)

test_data = numpy.array(test_data)

train_data_label = numpy.array(train_data_label)

test_data_label = numpy.array(test_data_label)

lr = 0.01
c1 = ConvolutionLayer(1, 6, 5, 5, 2, 1, learning_rate=lr)
r1 = ReLu()

m1 = MaxPoolingLayer(2, 2, 2)
c2 = ConvolutionLayer(6, 16, 5, 5, 0, 1, learning_rate=lr)
r2 = ReLu()
m2 = MaxPoolingLayer(2, 2, 2)
c3 = ConvolutionLayer(16, 120, 5, 5, 0, 1, learning_rate=lr)
r3 = ReLu()
f1 = Flatten()
fc1 = FullyConnectedLayer(120, 60, learning_rate=lr)
r4 = ReLu()
fc2 = FullyConnectedLayer(60, 2, learning_rate=lr)
s1 = Softmax()

layers = [c1, r1, m1, c2, r2, m2, c3, r3, f1, fc1, r4, fc2, s1]

cnn = CNN(layers)

#print('Training Lenet......')
cnn.train(training_data=train_data,training_label=train_data_label,batch_size=20,epoch=1)

#print('Testing Lenet......')
accuracy = cnn.test(data=test_data,label=test_data_label,test_size=200)
print('Testing accuracy:'+str(accuracy))


In [ ]:
#Write result into txt
f = open("CNN_result.txt", "w")
f.write('Testing accuracy:'+str(accuracy*100)+'%')
f.close()

In [ ]:
accuracy